In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from datetime import datetime, timedelta

import numpy as np 

import plotly.express as px
import plotly.graph_objects as go

import plotly

from operator import add

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train=pd.read_csv('../input/tabular-playground-series-mar-2022/train.csv')
display(train)
test=pd.read_csv('../input/tabular-playground-series-mar-2022/test.csv')
display(test)

### Convert time

In [ ]:
train["time"] = pd.to_datetime(train["time"])
train.info()

In [ ]:
train.describe()

In [ ]:
def create_ts_id(df):
    min_timestamp = df.time.min()
    def convert_to_timestamp_id(min_ts, cur_ts):
        seconds_from_start = cur_ts - min_ts

    df["ts_id"] = (df["time"] - train.time.min()) / (20*60)
    df["ts_id"] = df["ts_id"].dt.seconds.astype(int)
    return df

# Map congestion on position and direction

In [ ]:
def geo_spatial_detailed_df(orig_df):
    geo_df = orig_df.copy()
    #train["geo_ts_group"] = train.apply(lambda row: f'{row.x}_{row.y}_{row.ts_id}', axis=1)
    geo_df["geo_dir"] =  geo_df.apply(lambda row: f'{row.x}_{row.y}_{row.direction}', axis=1)
    # drop direction column to only have geo_dir categorical column
    geo_df.drop("direction", axis=1, inplace=True)
    # transform categorical to column
    geo_df = pd.get_dummies(geo_df)
    # set congestion for each geo_dir
    for c in geo_df.columns:
        if c.startswith("geo_dir"):
            geo_df[c] = geo_df[c] * geo_df.congestion
    # Drop unnecessary columns
    geo_df.drop("congestion", axis=1, inplace=True)
    geo_df.drop("row_id", axis=1, inplace=True)
    geo_df.drop("x", axis=1, inplace=True)
    geo_df.drop("y", axis=1, inplace=True)
    geo_df = geo_df.groupby(by="time").max()
    return geo_df

def geo_spatial_df(orig_df):
    geo_df = train.copy()
    geo_df.drop("direction", axis=1, inplace=True)
    geo_df.drop("row_id", axis=1, inplace=True)
    geo_df = geo_df.groupby(by=["time","x","y"]).sum()
    return geo_df

# Animate using plotly - group by area
Based on https://plotly.com/python/v3/gapminder-example/ and https://plotly.com/python/animations/

Only display a limited number of frames for memory consumption on web browser side

In [ ]:
geo_train = geo_spatial_df(train)
geo_train.info()

In [ ]:
geo_train.head(5)

In [ ]:
geo_train.describe()

In [ ]:
m,n = len(geo_train.index.levels[1]), len(geo_train.index.levels[2])
M = geo_train.values.reshape(-1,m,n).swapaxes(1,2)

In [ ]:
# Make the figure
config={'scrollZoom': False}
nb_frames_max = 100
frames = []
sliders_dict = {
    'active': 0,
    'yanchor': 'top',
    'xanchor': 'left',
    'currentvalue': {
        'font': {'size': 20},
        'prefix': 'time:',
        'visible': True,
        'xanchor': 'right'
    },
    'transition': {'duration': 300, 'easing': 'cubic-in-out'},
    'pad': {'b': 10, 't': 50},
    'len': 0.9,
    'x': 0.1,
    'y': 0,
    'steps': []
}
for idx, ts in enumerate(list(geo_train.index.levels[0])[:nb_frames_max]):
    if idx == 0:
        next
    frame = go.Frame(data=[go.Heatmap(z=M[idx])],name=str(ts))
    frames.append(frame)
    slider_step = {'args': [
        [str(ts)],
        {'frame': {'duration': 300, 'redraw': True},
         'mode': 'immediate',
       'transition': {'duration': 100}}
     ],
     'label': str(ts),
     'method': 'animate'}
    sliders_dict['steps'].append(slider_step)

fig = go.Figure(data=[go.Heatmap(z=M[0], zmin=0, zmax= 700)],
                   layout={
                       'sliders': [sliders_dict],
                       'xaxis': {'range': [-0.5, m-0.5], 'title': 'x-axis', 'gridcolor': '#FFFFFF', 'tickmode': 'linear', 'tick0': 0, 'dtick': 1},
                       'yaxis': {'range': [-0.5, n-0.5], 'title': 'y-axis', 'gridcolor': '#FFFFFF', 'tickmode': 'linear', 'tick0': 0, 'dtick': 1},
                       'plot_bgcolor': 'rgb(223, 232, 243)',
                       'updatemenus': [
                           {
                               'buttons': [
                                   {
                                        'args': [None, {'frame': {'duration': 300, 'redraw': True},
                                                 'fromcurrent': True, 'transition': {'duration': 100, 'easing': 'quadratic-in-out'}}],
                                        'label': 'Play',
                                        'method': 'animate'
                                    },
                                    {
                                        'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate',
                                        'transition': {'duration': 0}}],
                                        'label': 'Pause',
                                        'method': 'animate'
                                    }
                                ],
                                'direction': 'left',
                                'pad': {'r': 10, 't': 87},
                                'showactive': False,
                                'type': 'buttons',
                                'x': 0.1,
                                'xanchor': 'right',
                                'y': 0,
                                'yanchor': 'top'
                            }
                        ]
                   },
                   frames=frames
                  )

fig.show(config=config)


# Animate using plotly - with road directions
Based on https://plotly.com/python/v3/gapminder-example/ and https://plotly.com/python/animations/

Only display a limited number of frames for memory consumption on web browser side

In [ ]:
pad = 0.1
map_dir_offset = {
    'NB': {'x': [0+pad, 0+pad], 'y': [-0.5, 0.5]},
    'SB': {'x': [0-pad, 0-pad], 'y': [0.5, -0.5]},
    'EB': {'x': [-0.5, 0.5], 'y': [0-pad, 0-pad]},
    'WB': {'x': [-0.5, 0.5], 'y': [0+pad, 0+pad]},
    'NW': {'x': [0.5, -0.5], 'y': [-0.5+pad, 0.5+pad]},
    'SE': {'x': [-0.5, 0.5], 'y': [0.5-pad, -0.5-pad]},
    'NE': {'x': [-0.5, 0.5], 'y': [-0.5-pad, 0.5-pad]},
    'SW': {'x': [0.5, -0.5], 'y': [0.5+pad, -0.5+pad]}
}
def get_line_coords(x, y, direction):
    return list(map(add, [x,x], map_dir_offset[direction]['x'])), list(map(add, [y,y], map_dir_offset[direction]['y']))

In [ ]:
geo_train = geo_spatial_detailed_df(train)
#geo_train.info()

In [ ]:
def init_colorscale():
    bluered_colors, scale = plotly.colors.convert_colors_to_same_type(plotly.colors.sequential.Bluered)
    colorscale = plotly.colors.make_colorscale(bluered_colors, scale=scale)
    return colorscale
    
def get_color(value, cs, vmin=0, vmax=700):
    norm_value = float(value-vmin)/float(vmax)
    return get_continuous_color(cs, norm_value)
    
def get_continuous_color(colorscale, intermed):
    """
    From https://stackoverflow.com/a/64655638
    Plotly continuous colorscales assign colors to the range [0, 1]. This function computes the intermediate
    color for any value in that range.

    Plotly doesn't make the colorscales directly accessible in a common format.
    Some are ready to use:

        colorscale = plotly.colors.PLOTLY_SCALES["Greens"]

    Others are just swatches that need to be constructed into a colorscale:

        viridis_colors, scale = plotly.colors.convert_colors_to_same_type(plotly.colors.sequential.Viridis)
        colorscale = plotly.colors.make_colorscale(viridis_colors, scale=scale)

    :param colorscale: A plotly continuous colorscale defined with RGB string colors.
    :param intermed: value in the range [0, 1]
    :return: color in rgb string format
    :rtype: str
    """
    if len(colorscale) < 1:
        raise ValueError("colorscale must have at least one color")

    if intermed <= 0 or len(colorscale) == 1:
        return colorscale[0][1]
    if intermed >= 1:
        return colorscale[-1][1]

    for cutoff, color in colorscale:
        if intermed > cutoff:
            low_cutoff, low_color = cutoff, color
        else:
            high_cutoff, high_color = cutoff, color
            break

    # noinspection PyUnboundLocalVariable
    return plotly.colors.find_intermediate_color(
        lowcolor=low_color, highcolor=high_color,
        intermed=((intermed - low_cutoff) / (high_cutoff - low_cutoff)),
        colortype="rgb")

def get_infos_from_column_name(column_name):
    # column name on the form geo_dir_X_Y_direction
    values = column_name.split("_")
    x = int(values[2])
    y = int(values[3])
    direction = values[4]
    return x, y, direction

In [ ]:
cs = init_colorscale()

In [ ]:
# Make the figure
config={'scrollZoom': False}
nb_frames_max = 100
congestion_min = 0
congestion_max = 100
frames = []
sliders_dict = {
    'active': 0,
    'yanchor': 'top',
    'xanchor': 'left',
    'currentvalue': {
        'font': {'size': 20},
        'prefix': 'time:',
        'visible': True,
        'xanchor': 'right'
    },
    'transition': {'duration': 300, 'easing': 'cubic-in-out'},
    'pad': {'b': 10, 't': 50},
    'len': 0.9,
    'x': 0.1,
    'y': 0,
    'steps': []
}
for idx, ts in enumerate(list(geo_train.index)[:nb_frames_max]):
    scatters = [go.Scatter(x=[None],
                           y=[None],
                           mode='markers',
                           showlegend = False,
                           marker=dict(
                               colorscale=cs, 
                               showscale=True,
                               cmin=0,
                               cmax=100,
                               colorbar=dict(thickness=10, tickvals=[0, 100], outlinewidth=0)
                           ),
                           hoverinfo='none'
                          )]
    for c in geo_train.columns:
        x, y, direction = get_infos_from_column_name(c)
        congestion = geo_train.loc[ts, c]
        clr = get_color(congestion, cs, congestion_min, congestion_max)
        xcoords, ycoords = get_line_coords(x, y, direction)
        scatters.append(go.Scatter(x=xcoords, 
                                   y=ycoords, 
                                   line=dict(color=clr), 
                                   mode='lines',
                                   showlegend = False,
                                   hovertemplate = '',
                                   hoverinfo='skip'))
    if idx == 0:
        first_frame_scatters = scatters
    frame = go.Frame(data=scatters,name=str(ts))
    frames.append(frame)
    slider_step = {'args': [
        [str(ts)],
        {'frame': {'duration': 300, 'redraw': True},
         'mode': 'immediate',
       'transition': {'duration': 100}}
     ],
     'label': str(ts),
     'method': 'animate'}
    sliders_dict['steps'].append(slider_step)

fig = go.Figure(data=first_frame_scatters,
                   layout={
                       'sliders': [sliders_dict],
                       'xaxis': {'range': [-0.5, m-0.5], 'title': 'x-axis', 'gridcolor': '#FFFFFF', 'tickmode': 'linear', 'tick0': 0, 'dtick': 1},
                       'yaxis': {'range': [-0.5, n-0.5], 'title': 'y-axis', 'gridcolor': '#FFFFFF', 'tickmode': 'linear', 'tick0': 0, 'dtick': 1},
                       'plot_bgcolor': 'rgb(223, 232, 243)',
                       'updatemenus': [
                           {
                               'buttons': [
                                   {
                                        'args': [None, {'frame': {'duration': 300, 'redraw': True},
                                                 'fromcurrent': True, 'transition': {'duration': 100, 'easing': 'quadratic-in-out'}}],
                                        'label': 'Play',
                                        'method': 'animate'
                                    },
                                    {
                                        'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate',
                                        'transition': {'duration': 0}}],
                                        'label': 'Pause',
                                        'method': 'animate'
                                    }
                                ],
                                'direction': 'left',
                                'pad': {'r': 10, 't': 87},
                                'showactive': False,
                                'type': 'buttons',
                                'x': 0.1,
                                'xanchor': 'right',
                                'y': 0,
                                'yanchor': 'top'
                            }
                        ]
                   },
                   frames=frames
                  )

fig.show(config=config)